In [9]:
import csv
import random
import csv
import math
import os

prec = []
rec = []

def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
def str_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    uniqueName = set(class_values)
    traverse = dict()
    for i, value in enumerate(uniqueName):
        traverse[value] = i
    for row in dataset:
        row[column] = traverse[row[column]]
    return traverse

def cross_validation(dataset, n_folds):
    datasetSplit = list()
    datasetCopy = list(dataset)
    foldSize = int(len(dataset) / n_folds)
    for i in range(n_folds):
        folds = list()
        while len(folds) < foldSize:
            index = random.randrange(len(datasetCopy))
            folds.append(datasetCopy.pop(index))
        datasetSplit.append(folds)
    return datasetSplit

def accuracy(actual, predicted):
    true = 0
    for i in range(len(actual)):
        true_positive = 0
        false_positive = 0
        true_negative = 0
        false_negative = 0
        if actual[i] == predicted[i]:
            true += 1
        if actual[i] == 1 and predicted[i] == 1:
            true_positive += 1
        if actual[i] == 1 and predicted[i] == 0:
            false_negative += 1
        if actual[i] == 0 and predicted[i] == 1:
            false_positive += 1
        if actual[i] == 0 and predicted[i] == 0:
            true_negative += 1
        
    prec.append([true_positive, true_negative, false_positive, false_negative])
       
    return correct / float(len(actual)) * 100.0
def predictWeights(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0
def train_weights(train, l_rate, n_epoch):
    weights = [round(random.uniform(0, 1), 2) for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = predictWeights(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    return weights
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predictWeights(row, weights)
        predictions.append(prediction)
    return(predictions)
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy(actual, predicted)
        scores.append(accuracy)
    return scores

def main():
    filename = "Datasets/Heart_Dataset.csv"
    attributes = []
    dataset = []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            dataset.append(row)
    attributes = dataset[0]
    dataset = dataset[1:]
    for i in range(len(dataset[0])-1):
        str_to_float(dataset, i)
    str_to_int(dataset, len(dataset[0])-1)

    n_folds = 10
    l_rate = 0.3
    n_epoch = 500
    scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
    print('Scores:')
    for i in range(len(scores)):
        print(i+1, ": ", scores[i])
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    ttp = 0
    ttn = 0
    tfp = 0
    tfn = 0
    for i in range(len(prec)):
        ttp += prec[i][0]
        ttn += prec[i][1]
        tfp += prec[i][2]
        tfn += prec[i][3]
    print('Threshold: 0.0')
    print('Precision: ', ttp / (ttp + tfp))
    print('Recall: ', ttp / (ttp + tfn))

if __name__ == '__main__':
    main()

Scores:
1 :  85.71428571428571
2 :  85.71428571428571
3 :  42.857142857142854
4 :  85.71428571428571
5 :  57.14285714285714
6 :  57.14285714285714
7 :  71.42857142857143
8 :  71.42857142857143
9 :  57.14285714285714
10 :  57.14285714285714
Mean Accuracy: 67.143%
Threshold: 0.0
Precision:  0.6666666666666666
Recall:  0.4
